<a href="https://colab.research.google.com/github/GabOs098/Bioinformatica/blob/main/ver_final_TP_Final_Bioinform%C3%A1tica_Shimabukuro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo práctico final
(Por Gabriel Shimabukuro)

El trabajo práctico tomará como referencia la actividad relacionada con los modelos ocultos de Markov. En mi caso usaremos el HMM (Modelo oculto de Markov) y el algoritmo de Viterbi para detectar los exones e intrones del gen. Trataremos de generar unas probabilidades iniciales a partir de una gen en particular y luego analizaremos distintas variantes del mismo gen. Luego, usaremos el algoritmo de Baum-Welch para generar nuevas matrices de probabilidades de transición y de emisión.

El gen que elegí para trabajar es el que codifica a la catalasa. La catalasa es una enzima muy importante en la protección celular, ya que se encarga de descomponer el peróxido de hidrógeno (H2O2), una sustancia tóxica generada durante el metabolismo celular, convirtiendola en agua(H2O) y oxígeno(O2).
Podemos encontrar la secuencia de nucleótidos del gen en el GenBank, una base de datos de secuencias genéticas del NIH (National Institutes of Health) de Estados Unidos. El siguiente link redirige a la página Web que hace referencia a la catalasa:
[Archivo del GenBank de la catalasa](https://www.ncbi.nlm.nih.gov/nuccore/NG_013339.2?report=genbank).

En la página anterior podemos encontrar el gen completo de la catalasa, es decir, sus exones e intrones. Si recordamos la secuencia de inicio de un gen, AUG en el ARNm, que sería ATG en la cadena ADN, podemos encontrarla en la posición 5090. Estos primeros nucleótidos corresponden al promotor y, en este caso, no los tendremos en cuenta. Este desfase de 5090 nucléotidos lo usaremos para marcar los exones junto con la información que nos brinda la página.

desfase(5090) + index de la web = posición exón en mi secuencia





In [ ]:
#Cambiar entorno de ejecución a R
install.packages("seqinr")
install.packages("HMM")
library(seqinr)
library('HMM')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# FUNCIONES

toCodonSeq <- function(seq) {
  codon_seq = c()
  for (i in seq(1, length(seq), by=3)) {
    codon = paste(gen_seq[i], gen_seq[sum(i + 1)], gen_seq[sum(i + 2)])
    codon_seq = c(codon_seq, codon)
  }

  return(codon_seq)
}

# state = Es el estado que queremos generar
# start & end = de la secuencia del genBank
generateStates <- function(start, end, state) {
  states_seq = c()
  start_exon = 1
  end_exon = sum(end - start, 1)
  exon_len = end_exon - start_exon
  n_codones = exon_len/3 + 1
  for(i in 1:n_codones){
    states_seq = c(states_seq, state)
  }

  return(states_seq)
}

# codones = secuencia de tripletes
# index = indice de AUG (cantidad de nucleotidos desde inicio hasta AUG), desfase hasta AUG
# start & end = hacen referencia a la secuencia de nucleotidos no de codones
findCodon <- function(codones, index, start, end) {
  st_cn = sum((start - index)/3, 1)
  end_cn = sum((end - index)/3, 1)

  return(codones[st_cn:end_cn])
}


# Crea vector de probabilidades de emisión de codones a partir de una secuencia de codones
generate_emission_prob <- function(cod_sec) {

  #Inicializacion contadores
  aaa = 0
  aac = 0
  aag = 0
  aat = 0
  aca = 0
  acc = 0
  acg = 0
  act = 0
  aga = 0
  agc = 0
  agg = 0
  agt = 0
  ata = 0
  atc = 0
  atg = 0
  att = 0
  caa = 0
  cac = 0
  cag = 0
  cat = 0
  cca = 0
  ccc = 0
  ccg = 0
  cct = 0
  cga = 0
  cgc = 0
  cgg = 0
  cgt = 0
  cta = 0
  ctc = 0
  ctg = 0
  ctt = 0
  gaa = 0
  gac = 0
  gag = 0
  gat = 0
  gca = 0
  gcc = 0
  gcg = 0
  gct = 0
  gga = 0
  ggc = 0
  ggg = 0
  ggt = 0
  gta = 0
  gtc = 0
  gtg = 0
  gtt = 0
  taa = 0
  tac = 0
  tag = 0
  tat = 0
  tca = 0
  tcc = 0
  tcg = 0
  tct = 0
  tga = 0
  tgc = 0
  tgg = 0
  tgt = 0
  tta = 0
  ttc = 0
  ttg = 0
  ttt = 0

  for (i in cod_sec) {
    if (i == "a a a") {
      aaa = sum(aaa, 1)
    }
    if (i == "a a c") {
      aac = sum(aac, 1)
    }
    if (i == "a a g") {
      aag = sum(aag, 1)
    }
    if (i == "a a t") {
      aat = sum(aat, 1)
    }
    if (i == "a c a") {
      aca = sum(aca, 1)
    }
    if (i == "a c c") {
      acc = sum(acc, 1)
    }
    if (i == "a c g") {
      acg = sum(acg, 1)
    }
    if (i == "a c t") {
      act = sum(act, 1)
    }
    if (i == "a g a") {
      aga = sum(aga, 1)
    }
    if (i == "a g c") {
      agc = sum(agc, 1)
    }
    if (i == "a g g") {
      agg = sum(agg, 1)
    }
    if (i == "a g t") {
      agt = sum(agt, 1)
    }
    if (i == "a t a") {
      ata = sum(ata, 1)
    }
    if (i == "a t c") {
      atc = sum(atc, 1)
    }
    if (i == "a t g") {
      atg = sum(atg, 1)
    }
    if (i == "a t t") {
      att = sum(att, 1)
    }
    if (i == "c a a") {
      caa = sum(caa, 1)
    }
    if (i == "c a c") {
      cac = sum(cac, 1)
    }
    if (i == "c a g") {
      cag = sum(cag, 1)
    }
    if (i == "c a t") {
      cat = sum(cat, 1)
    }
    if (i == "c c a") {
      cca = sum(cca, 1)
    }
    if (i == "c c c") {
      ccc = sum(ccc, 1)
    }
    if (i == "c c g") {
      ccg = sum(ccg, 1)
    }
    if (i == "c c t") {
      cct = sum(cct, 1)
    }
    if (i == "c g a") {
      cga = sum(cga, 1)
    }
    if (i == "c g c") {
      cgc = sum(cgc, 1)
    }
    if (i == "c g g") {
      cgg = sum(cgg, 1)
    }
    if (i == "c g t") {
      cgt = sum(cgt, 1)
    }
    if (i == "c t a") {
      cta = sum(cta, 1)
    }
    if (i == "c t c") {
      ctc = sum(ctc, 1)
    }
    if (i == "c t g") {
      ctg = sum(ctg, 1)
    }
    if (i == "c t t") {
      ctt = sum(ctt, 1)
    }
    if (i == "g a a") {
      gaa = sum(gaa, 1)
    }
    if (i == "g a c") {
      gac = sum(gac, 1)
    }
    if (i == "g a g") {
      gag = sum(gag, 1)
    }
    if (i == "g a t") {
      gat = sum(gat, 1)
    }
    if (i == "g c a") {
      gca = sum(gca, 1)
    }
    if (i == "g c c") {
      gcc = sum(gcc, 1)
    }
    if (i == "g c g") {
      gcg = sum(gcg, 1)
    }
    if (i == "g c t") {
      gct = sum(gct, 1)
    }
    if (i == "g g a") {
      gga = sum(gga, 1)
    }
    if (i == "g g c") {
      ggc = sum(ggc, 1)
    }
    if (i == "g g g") {
      ggg = sum(ggg, 1)
    }
    if (i == "g g t") {
      ggt = sum(ggt, 1)
    }
    if (i == "g t a") {
      gta = sum(gta, 1)
    }
    if (i == "g t c") {
      gtc = sum(gtc, 1)
    }
    if (i == "g t g") {
      gtg = sum(gtg, 1)
    }
    if (i == "g t t") {
      gtt = sum(gtt, 1)
    }
    if (i == "t a a") {
      taa = sum(taa, 1)
    }
    if (i == "t a c") {
      tac = sum(tac, 1)
    }
    if (i == "t a g") {
      tag = sum(tag, 1)
    }
    if (i == "t a t") {
      tat = sum(tat, 1)
    }
    if (i == "t c a") {
      tca = sum(tca, 1)
    }
    if (i == "t c c") {
      tcc = sum(tcc, 1)
    }
    if (i == "t c g") {
      tcg = sum(tcg, 1)
    }
    if (i == "t c t") {
      tct = sum(tct, 1)
    }
    if (i == "t g a") {
      tga = sum(tga, 1)
    }
    if (i == "t g c") {
      tgc = sum(tgc, 1)
    }
    if (i == "t g g") {
      tgg = sum(tgg, 1)
    }
    if (i == "t g t") {
      tgt = sum(tgt, 1)
    }
    if (i == "t t a") {
      tta = sum(tta, 1)
    }
    if (i == "t t c") {
      ttc = sum(ttc, 1)
    }
    if (i == "t t g") {
      ttg = sum(ttg, 1)
    }
    if (i == "t t t") {
      ttt = sum(ttt, 1)
    }
  }

  total_codons = length(cod_sec)

  #Probabilidades
  prob_aaa = aaa / total_codons
  prob_aac = aac / total_codons
  prob_aag = aag / total_codons
  prob_aat = aat / total_codons
  prob_aca = aca / total_codons
  prob_acc = acc / total_codons
  prob_acg = acg / total_codons
  prob_act = act / total_codons
  prob_aga = aga / total_codons
  prob_agc = agc / total_codons
  prob_agg = agg / total_codons
  prob_agt = agt / total_codons
  prob_ata = ata / total_codons
  prob_atc = atc / total_codons
  prob_atg = atg / total_codons
  prob_att = att / total_codons
  prob_caa = caa / total_codons
  prob_cac = cac / total_codons
  prob_cag = cag / total_codons
  prob_cat = cat / total_codons
  prob_cca = cca / total_codons
  prob_ccc = ccc / total_codons
  prob_ccg = ccg / total_codons
  prob_cct = cct / total_codons
  prob_cga = cga / total_codons
  prob_cgc = cgc / total_codons
  prob_cgg = cgg / total_codons
  prob_cgt = cgt / total_codons
  prob_cta = cta / total_codons
  prob_ctc = ctc / total_codons
  prob_ctg = ctg / total_codons
  prob_ctt = ctt / total_codons
  prob_gaa = gaa / total_codons
  prob_gac = gac / total_codons
  prob_gag = gag / total_codons
  prob_gat = gat / total_codons
  prob_gca = gca / total_codons
  prob_gcc = gcc / total_codons
  prob_gcg = gcg / total_codons
  prob_gct = gct / total_codons
  prob_gga = gga / total_codons
  prob_ggc = ggc / total_codons
  prob_ggg = ggg / total_codons
  prob_ggt = ggt / total_codons
  prob_gta = gta / total_codons
  prob_gtc = gtc / total_codons
  prob_gtg = gtg / total_codons
  prob_gtt = gtt / total_codons
  prob_taa = taa / total_codons
  prob_tac = tac / total_codons
  prob_tag = tag / total_codons
  prob_tat = tat / total_codons
  prob_tca = tca / total_codons
  prob_tcc = tcc / total_codons
  prob_tcg = tcg / total_codons
  prob_tct = tct / total_codons
  prob_tga = tga / total_codons
  prob_tgc = tgc / total_codons
  prob_tgg = tgg / total_codons
  prob_tgt = tgt / total_codons
  prob_tta = tta / total_codons
  prob_ttc = ttc / total_codons
  prob_ttg = ttg / total_codons
  prob_ttt = ttt / total_codons

  vector_prob = c( prob_aaa, prob_aac, prob_aag, prob_aat,
    prob_aca, prob_acc, prob_acg, prob_act,
    prob_aga, prob_agc, prob_agg, prob_agt,
    prob_ata, prob_atc, prob_atg, prob_att,
    prob_caa, prob_cac, prob_cag, prob_cat,
    prob_cca, prob_ccc, prob_ccg, prob_cct,
    prob_cga, prob_cgc, prob_cgg, prob_cgt,
    prob_cta, prob_ctc, prob_ctg, prob_ctt,
    prob_gaa, prob_gac, prob_gag, prob_gat,
    prob_gca, prob_gcc, prob_gcg, prob_gct,
    prob_gga, prob_ggc, prob_ggg, prob_ggt,
    prob_gta, prob_gtc, prob_gtg, prob_gtt,
    prob_taa, prob_tac, prob_tag, prob_tat,
    prob_tca, prob_tcc, prob_tcg, prob_tct,
    prob_tga, prob_tgc, prob_tgg, prob_tgt,
    prob_tta, prob_ttc, prob_ttg, prob_ttt
  )

  dif_cod = c(
    "aaa", "aac", "aag", "aat",
    "aca", "acc", "acg", "act",
    "aga", "agc", "agg", "agt",
    "ata", "atc", "atg", "att",
    "caa", "cac", "cag", "cat",
    "cca", "ccc", "ccg", "cct",
    "cga", "cgc", "cgg", "cgt",
    "cta", "ctc", "ctg", "ctt",
    "gaa", "gac", "gag", "gat",
    "gca", "gcc", "gcg", "gct",
    "gga", "ggc", "ggg", "ggt",
    "gta", "gtc", "gtg", "gtt",
    "taa", "tac", "tag", "tat",
    "tca", "tcc", "tcg", "tct",
    "tga", "tgc", "tgg", "tgt",
    "tta", "ttc", "ttg", "ttt"
  )


  m = rbind(dif_cod, vector_prob)
  print(m)

  return (vector_prob)
}


generate_emission_matrix <- function(probs_1, probs_2, probs_3) {

  combination = c()
  for (i in 1:length(probs_1)) {
    combination = c(combination, probs_1[i], probs_2[i], probs_3[i])
  }

  m = matrix(combination, nrow = 3, ncol=64)
  print(m)

  return (m)
}


# Reacomoda el índice de la secuencia de nuclétidos según deseemos restar tripletes
rest_index <- function(index_secuences, cant_triplete, desfase) {

  new_index_sec = c()
  div = cant_triplete * 3

  if(index_secuences[1] == desfase) {
    new_index_sec = c(new_index_sec, index_secuences[1])
    index_secuences = index_secuences[2:length(index_secuences)]
  }

  for(i in index_secuences) {

    index_triplete = (((i + 1) - desfase) / div)
    decimal = index_triplete - floor(index_triplete)

    if(decimal == 0) {
      new_index = i - 3
    }
    if(round(decimal,1) == 0.3) {
      new_index = i - 4
    }
    if(round(decimal,1) == 0.7) {
      new_index = i - 5
    }

    new_index_sec = c(new_index_sec, new_index)
  }

  return (new_index_sec)
}

#exons_end = c(5155, 15439, 17174, 18283, 19270, 20002, 22278, 22893, 27465, 30311, 34471, 37117, 37510)
# Reacomoda el índice de la secuencia de nuclétidos según deseemos sumar tripletes
sum_index <- function(index_secuences, cant_triplete, desfase) {

  new_index_sec = c()
  div = cant_triplete * 3

  for(i in index_secuences) {

    index_triplete = (((i + 1) - desfase) / div)
    decimal = index_triplete - floor(index_triplete)

    if(decimal == 0) {
      new_index = sum(i, 3)
    }
    if(round(decimal,1) == 0.3) {
      new_index = sum(i, 5)
    }
    if(round(decimal,1) == 0.7) {
      new_index = sum(i, 4)
    }

    new_index_sec = c(new_index_sec, new_index)
  }

  return (new_index_sec)
}


# sec_index_of_starts = Vector con los indices que indican el comienzo de un exón, intron o zona de splicing
# sec_index_of_ends = Vector con los indices que indican el final de un exón, intron o zona de splicing
extract_sec <- function(sec_index_of_starts, sec_index_of_ends, str_state) {

  states_all_secuencies = c()
  cod_all_secuencies = c()

  if(length(sec_index_of_starts) != length(sec_index_of_ends)) {
    print("Los vectores de inicio y final no tienen la misma cantidad de secuencias")
    return (c())
  }

  for(i in 1:length(sec_index_of_starts)) {
    start = as.integer(sec_index_of_starts[i])
    end = as.integer(sec_index_of_ends[i])
    aux_state = generateStates(start, end, str_state)
    aux_cod_exon = findCodon(codons, 5090, start, end)
    states_all_secuencies = c(states_all_secuencies, aux_state)
    cod_all_secuencies = c(cod_all_secuencies, aux_cod_exon)
  }

  length(states_all_secuencies)
  length(cod_all_secuencies)
  m = rbind(states_all_secuencies, cod_all_secuencies)
  print(m)

  return(list(states = states_all_secuencies, triplets = cod_all_secuencies))
}


[secuencia con la que trabajamos](https://www.ncbi.nlm.nih.gov/nuccore/NG_013339.2?report=genbank)

In [ ]:
seqinr_file = seqinr::read.fasta(file = "sequence_CAT.fasta", seqtype = "DNA")
complete_seq = seqinr::getSequence(seqinr_file[1])

gen_len = seqinr::getLength(complete_seq)
idx_aug = 5090
idx_tga = 37510

gen_seq = complete_seq[[1]][idx_aug:idx_tga]     #Primer exon, empiezo en CDS, me falta region UTR, para facilitar el marco de lectura antes es promotor, En el primer exon encontramos AUG, termina en TGA
head(gen_seq)
tail(gen_seq)

codons = toCodonSeq(gen_seq)
head(codons)
tail(codons)







[1] "a" "t" "g" "g" "c" "t"

[1] "t" "g" "t" "g" "a" "g"

[1] "a t g" "g c t" "g a c" "a g c" "c g g" "g a t"

[1] "a g a" "a g g" "c a a" "a t c" "t g t" "g a g"

In [ ]:
#Primer exon, genero la cantidad de estados E correspondientes a la cantidad de tripletes
st_exon_1 = generateStates(5090, 5155, 'E')
cod_exon_1 = findCodon(codons, 5090, 5090, 5155)
print(length(st_exon_1))
print(length(cod_exon_1))

# Matriz de estado y secuencia observada
m = rbind(st_exon_1, cod_exon_1)
print(m)

[1] 22
[1] 22
           [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_1  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_1 "a t g" "g c t" "g a c" "a g c" "c g g" "g a t" "c c c" "g c c"
           [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_1  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_1 "a g c" "g a c" "c a g" "a t g" "c a g" "c a c" "t g g" "a a g"
           [,17]   [,18]   [,19]   [,20]   [,21]   [,22]  
st_exon_1  "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_1 "g a g" "c a g" "c g g" "g c c" "g c g" "c a g"


In [ ]:
#Intron
st_intron_1 = generateStates(5156, 15265, 'I')
cod_intron_1 = findCodon(codons, 5090, 5156, 15265)
print(length(st_intron_1))
print(length(cod_intron_1))
head(cod_intron_1)
tail(cod_intron_1)

[1] 3370
[1] 3370


[1] "g t a" "c a c" "t c t" "g t g" "c t c" "c c c"

[1] "t t c" "t t t" "c t g" "t g t" "t c c" "t g t"

In [ ]:
# El triplete forma parte del exon e intron, 3er estado S, en este punto los tripletes
st_splicing_1 = 'S'
cod_splicing_1 = findCodon(codons, 5090, 15266, 15268)

m3 = rbind(st_splicing_1, cod_splicing_1)
print(m3)

               [,1]   
st_splicing_1  "S"    
cod_splicing_1 "a g a"


In [ ]:
# Como se me movio el marco de lectura, creo q los tripletes ya no son codones
#Segundo exon
st_exon_2 = generateStates(15269, 15439, 'E')
cod_exon_2 = findCodon(codons, 5090, 15269, 15439)
print(length(st_exon_2))
print(length(cod_exon_2))

m4 = rbind(st_exon_2, cod_exon_2)
print(m4)

[1] 57
[1] 57
           [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_2  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_2 "a a g" "c t g" "a t g" "t c c" "t g a" "c c a" "c t g" "g a g"
           [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_2  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_2 "c t g" "g t a" "a c c" "c a g" "t a g" "g a g" "a c a" "a a c"
           [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
st_exon_2  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_2 "t t a" "a t g" "t t a" "t t a" "c a g" "t a g" "g g c" "c c c"
           [,25]   [,26]   [,27]   [,28]   [,29]   [,30]   [,31]   [,32]  
st_exon_2  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_2 "g t g" "g g c" "c c c" "t t c" "t t g" "t t c" "a g g" "a t g"
           [,33]   [,34]   [,35]   [,36]   [,37]   [,38]   [,39]   [,40]  
st_exon_2  

In [ ]:
#Intron
st_intron_2 = generateStates(15440, 17062, 'I')
cod_intron_2 = findCodon(codons, 5090, 15440, 17062)
print(length(st_intron_2))
print(length(cod_intron_2))

head(cod_intron_2)
tail(cod_intron_2)
#m5 = rbind(st_intron_2, cod_intron_2)
#print(m5)

[1] 541
[1] 541


[1] "g t a" "a g t" "g c t" "g t g" "t t t" "a t t"

[1] "t c t" "g t g" "t c t" "t t c" "t c g" "t t a"

In [ ]:
#Splicing
st_splicing_2 = 'S'
cod_splicing_2 = findCodon(codons, 5090, 17063, 17065)
print(length(st_splicing_2))
print(length(cod_splicing_2))

m6 = rbind(st_splicing_2, cod_splicing_2)
print(m6)

[1] 1
[1] 1
               [,1]   
st_splicing_2  "S"    
cod_splicing_2 "g g g"


In [ ]:
# Exon 3
st_exon_3 = generateStates(17066, 17173, 'E')
cod_exon_3 = findCodon(codons, 5090, 17066, 17173)
print(length(st_exon_3))
print(length(cod_exon_3))

m7 = rbind(st_exon_3, cod_exon_3)
print(m7)

[1] 36
[1] 36
           [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_3  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_3 "g c c" "t t t" "g g c" "t a c" "t t t" "g a g" "g t c" "a c a"
           [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_3  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_3 "c a t" "g a c" "a t t" "a c c" "a a a" "t a c" "t c c" "a a g"
           [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
st_exon_3  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_3 "g c a" "a a g" "g t a" "t t t" "g a g" "c a t" "a t t" "g g a"
           [,25]   [,26]   [,27]   [,28]   [,29]   [,30]   [,31]   [,32]  
st_exon_3  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_3 "a a g" "a a g" "a c t" "c c c" "a t c" "g c a" "g t t" "c g g"
           [,33]   [,34]   [,35]   [,36]  
st_exon_3  "E"     "E"     "E"     "E"    


In [ ]:
# Splicing
st_splicing_3 = 'S'
cod_splicing_3 = findCodon(codons, 5090, 17174, 17176)
print(length(st_splicing_3))
print(length(cod_splicing_3))

m8 = rbind(st_splicing_3, cod_splicing_3)
print(m8)

[1] 1
[1] 1
               [,1]   
st_splicing_3  "S"    
cod_splicing_3 "g g t"


In [ ]:
# Intron
st_intron_3 = generateStates(17177, 18151, 'I')
cod_intron_3 = findCodon(codons, 5090, 17177, 18151)
print(length(st_intron_3))
print(length(cod_intron_3))
head(cod_intron_3)
tail(cod_intron_3)

#m9 = rbind(states9, aux9)
#print(m9)

[1] 325
[1] 325


[1] "a a g" "t t g" "g t t" "t a t" "t g g" "c g t"

[1] "t t c" "c a t" "t t g" "a a t" "a t t" "g t a"

In [ ]:
#Splicing
st_splicing_4 = generateStates(18152, 18154, 'S')
cod_splicing_4 = findCodon(codons, 5090, 18152, 18154)
print(length(st_splicing_4))
print(length(cod_splicing_4))

m10 = rbind(st_splicing_4, cod_splicing_4)
print(m10)

[1] 1
[1] 1
               [,1]   
st_splicing_4  "S"    
cod_splicing_4 "g c t"


In [ ]:
# Exon 4
st_exon_4 = generateStates(18155, 18283, 'E')
cod_exon_4 = findCodon(codons, 5090, 18155, 18283)
print(length(st_exon_4))
print(length(cod_exon_4))

m11 = rbind(st_exon_4, cod_exon_4)
print(m11)

[1] 43
[1] 43
           [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_4  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_4 "g g a" "g a a" "t c g" "g g t" "t c a" "g c t" "g a c" "a c a"
           [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_4  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_4 "g t t" "c g g" "g a c" "c c t" "c g t" "g g g" "t t t" "g c a"
           [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
st_exon_4  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_4 "g t g" "a a a" "t t t" "t a c" "a c a" "g a a" "g a t" "g g t"
           [,25]   [,26]   [,27]   [,28]   [,29]   [,30]   [,31]   [,32]  
st_exon_4  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_4 "a a c" "t g g" "g a t" "c t c" "g t t" "g g a" "a a t" "a a c"
           [,33]   [,34]   [,35]   [,36]   [,37]   [,38]   [,39]   [,40]  
st_exon_4  

In [ ]:
#Intron
st_intron_4 = generateStates(18284, 19165, 'I')
cod_intron_4 = findCodon(codons, 5090, 18284, 19165)
print(length(cod_intron_4))
print(length(cod_intron_4))
head(cod_intron_4)
tail(cod_intron_4)

#m12 = rbind(states12, aux12)
#print(m12)

[1] 294
[1] 294


[1] "g t a" "g g t" "a a t" "a g a" "g t a" "t t t"

[1] "t c t" "c t t" "t t t" "t c t" "a t t" "t a g"

In [ ]:
#Exon
st_exon_5 = generateStates(19166, 19270, 'E')
cod_exon_5 = findCodon(codons, 5090, 19166, 19270)
print(length(st_exon_5))
print(length(cod_exon_5))

m13 = rbind(st_exon_5, cod_exon_5)
print(m13)

[1] 35
[1] 35
           [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_5  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_5 "t t t" "c c a" "t c t" "t t t" "a t c" "c a c" "a g c" "c a a"
           [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_5  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_5 "a a g" "a g a" "a a t" "c c t" "c a g" "a c a" "c a t" "c t g"
           [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
st_exon_5  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_5 "a a g" "g a t" "c c g" "g a c" "a t g" "g t c" "t g g" "g a c"
           [,25]   [,26]   [,27]   [,28]   [,29]   [,30]   [,31]   [,32]  
st_exon_5  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_5 "t t c" "t g g" "a g c" "c t a" "c g t" "c c t" "g a g" "t c t"
           [,33]   [,34]   [,35]  
st_exon_5  "E"     "E"     "E"    
cod_exon_5 "c t 

In [ ]:
#intron
st_intron_5 = generateStates(19271, 19876, 'I')
cod_intron_5 = findCodon(codons, 5090, 19271, 19876)
print(length(st_intron_5))
print(length(cod_intron_5))
head(cod_intron_5)
tail(cod_intron_5)
#m14 = rbind(states14, aux14)
#print(m14)

[1] 202
[1] 202


[1] "g t a" "t g a" "a c c" "c t t" "t t t" "t g c"

[1] "t a t" "a t t" "t c t" "g t t" "c t t" "t a g"

In [ ]:
#exon
st_exon_6 = generateStates(19877, 20002, 'E')
cod_exon_6 = findCodon(codons, 5090, 19877, 20002)
print(length(st_exon_6))
print(length(cod_exon_6))

m15 = rbind(st_exon_6, cod_exon_6)
print(m15)

[1] 42
[1] 42
           [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_6  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_6 "g t t" "t c t" "t t c" "t t g" "t t c" "a g t" "g a t" "c g g"
           [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_6  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_6 "g g g" "a t t" "c c a" "g a t" "g g a" "c a t" "c g c" "c a c"
           [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
st_exon_6  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_6 "a t g" "a a t" "g g a" "t a t" "g g a" "t c a" "c a t" "a c t"
           [,25]   [,26]   [,27]   [,28]   [,29]   [,30]   [,31]   [,32]  
st_exon_6  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_6 "t t c" "a a g" "c t g" "g t t" "a a t" "g c a" "a a t" "g g g"
           [,33]   [,34]   [,35]   [,36]   [,37]   [,38]   [,39]   [,40]  
st_exon_6  

In [ ]:
#intron
st_intron_6 = generateStates(20003, 22084, 'I')
cod_intron_6 = findCodon(codons, 5090, 20003, 22084)
print(length(st_intron_6))
print(length(cod_intron_6))
head(cod_intron_6)
tail(cod_intron_6)

#m15 = rbind(states15, aux15)
#print(m15)

[1] 694
[1] 694


[1] "g t a" "t g t" "g t t" "a c c" "t t t" "g g g"

[1] "t t c" "t t t" "c t t" "t c a" "t t t" "t g t"

In [ ]:
#Splicing state
st_splicing_5 = 'S'
cod_splicing_5 = findCodon(codons, 5090, 22085, 22087)
print(length(st_splicing_5))
print(length(cod_splicing_5))

m17 = rbind(st_splicing_5, cod_splicing_5)
print(m17)

[1] 1
[1] 1
               [,1]   
st_splicing_5  "S"    
cod_splicing_5 "a g a"


In [ ]:
#Exon
st_exon_7 = generateStates(22088, 22276, 'E')
cod_exon_7 = findCodon(codons, 5090, 22088, 22276)
print(length(st_exon_7))
print(length(cod_exon_7))

m18 = rbind(st_exon_7, cod_exon_7)
print(m18)

[1] 63
[1] 63
           [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_7  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_7 "c t g" "a c c" "a g g" "g c a" "t c a" "a a a" "a c c" "t t t"
           [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_7  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_7 "c t g" "t t g" "a a g" "a t g" "c g g" "c g a" "g a c" "t t t"
           [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
st_exon_7  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_7 "c c c" "a g g" "a a g" "a t c" "c t g" "a c t" "a t g" "g c a"
           [,25]   [,26]   [,27]   [,28]   [,29]   [,30]   [,31]   [,32]  
st_exon_7  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_7 "t c c" "g g g" "a t c" "t t t" "t t a" "a c g" "c c a" "t t g"
           [,33]   [,34]   [,35]   [,36]   [,37]   [,38]   [,39]   [,40]  
st_exon_7  

In [ ]:
#Splicing state
st_splicing_6 = 'S'
cod_splicing_6 = findCodon(codons, 5090, 22277, 22279)
print(length(st_splicing_6))
print(length(cod_splicing_6))

m19 = rbind(st_splicing_6, cod_splicing_6)
print(m19)

[1] 1
[1] 1
               [,1]   
st_splicing_6  "S"    
cod_splicing_6 "a g g"


In [ ]:
#Intron
st_intron_7 = generateStates(22280, 22738, 'I')
cod_intron_7 = findCodon(codons, 5090, 22280, 22738)
print(length(st_intron_7))
print(length(cod_intron_7))
head(cod_intron_7)
tail(cod_intron_7)

#m20 = rbind(states20, aux20)
#print(m20)

[1] 153
[1] 153


[1] "t g a" "g t c" "a g t" "a a a" "c a a" "c t a"

[1] "g a a" "t a t" "g a c" "a t c" "a t t" "t t c"

In [ ]:
#Splicing state
st_splicing_7 = 'S'
cod_splicing_7 = findCodon(codons, 5090, 22739, 22741)
print(length(st_splicing_7))
print(length(cod_splicing_7))

m21 = rbind(st_splicing_7, cod_splicing_7)
print(m21)

[1] 1
[1] 1
               [,1]   
st_splicing_7  "S"    
cod_splicing_7 "a g g"


In [ ]:
#Exon
st_exon_8 = generateStates(22742, 22891, 'E')
cod_exon_8 = findCodon(codons, 5090, 22742, 22891)
print(length(st_exon_8))
print(length(cod_exon_8))

m22 = rbind(st_exon_8, cod_exon_8)
print(m22)

[1] 50
[1] 50
           [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_8  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_8 "t t t" "g g c" "c t c" "a c a" "a g g" "a c t" "a c c" "c t c"
           [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_8  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_8 "t c a" "t c c" "c a g" "t t g" "g t a" "a a c" "t g g" "t c t"
           [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
st_exon_8  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_8 "t a a" "a c c" "g g a" "a t c" "c a g" "t t a" "a t t" "a c t"
           [,25]   [,26]   [,27]   [,28]   [,29]   [,30]   [,31]   [,32]  
st_exon_8  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_8 "t t g" "c t g" "a g g" "t t g" "a a c" "a g a" "t a g" "c c t"
           [,33]   [,34]   [,35]   [,36]   [,37]   [,38]   [,39]   [,40]  
st_exon_8  

In [ ]:
#Splicing state
st_splicing_8 = 'S'
cod_splicing_8 = findCodon(codons, 5090, 22892, 22894)
print(length(st_splicing_8 ))
print(length(cod_splicing_8))

m23 = rbind(st_splicing_8, cod_splicing_8)
print(m23)

[1] 1
[1] 1
               [,1]   
st_splicing_8  "S"    
cod_splicing_8 "a g g"


In [ ]:
#Intron
st_intron_8 = generateStates(22895, 27325, 'I')
cod_intron_8 = findCodon(codons, 5090, 22895, 27325)
print(length(st_intron_8))
print(length(cod_intron_8))
head(cod_intron_8)
tail(cod_intron_8)

#m23 = rbind(states23, aux23)
#print(m23)

[1] 1477
[1] 1477


[1] "t g a" "g c c" "t g g" "t g g" "a t t" "g a g"

[1] "a t t" "g t a" "t t t" "a t t" "a c t" "g c a"

In [ ]:
#Splicing state
st_splicing_9 = 'S'
cod_splicing_9 = findCodon(codons, 5090, 27326, 27328)
print(length(st_splicing_9))
print(length(cod_splicing_9))

m25 = rbind(st_splicing_9, cod_splicing_9)
print(m25)

[1] 1
[1] 1
               [,1]   
st_splicing_9  "S"    
cod_splicing_9 "g g g"


In [ ]:
#Exon
st_exon_9 = generateStates(27329, 27463, 'E')
cod_exon_9 = findCodon(codons, 5090, 27329, 27463)
print(length(st_exon_9))
print(length(cod_exon_9))

m26 = rbind(st_exon_9, cod_exon_9)
print(m26)

[1] 45
[1] 45
           [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_9  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_9 "c c g" "c c t" "t t t" "t g c" "c t a" "t c c" "t g a" "c a c"
           [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_9  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_9 "t c a" "c c g" "c c a" "t c g" "c c t" "g g g" "a c c" "c a a"
           [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
st_exon_9  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_9 "t t a" "t c t" "t c a" "t a t" "a c c" "t g t" "g a a" "c t g"
           [,25]   [,26]   [,27]   [,28]   [,29]   [,30]   [,31]   [,32]  
st_exon_9  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_9 "t c c" "c t a" "c c g" "t g c" "t c g" "a g t" "g g c" "c a a"
           [,33]   [,34]   [,35]   [,36]   [,37]   [,38]   [,39]   [,40]  
st_exon_9  

In [ ]:
#Splicing state
st_splicing_10 = 'S'
cod_splicing_10 = findCodon(codons, 5090, 27464, 27466)
print(length(st_splicing_10))
print(length(cod_splicing_10))

m27 = rbind(st_splicing_10, cod_splicing_10)
print(m27)

[1] 1
[1] 1
                [,1]   
st_splicing_10  "S"    
cod_splicing_10 "g g g"


In [ ]:
#Intron
st_intron_9 = generateStates(27467, 30178, 'I')
cod_intron_9 = findCodon(codons, 5090, 27467, 30178)
print(length(st_intron_9))
print(length(cod_intron_9))
head(cod_intron_9)
tail(cod_intron_9)

#m28 = rbind(states28, aux28)
#print(m28)

[1] 904
[1] 904


[1] "t a g" "g c c" "t a a" "a g a" "c g t" "t g g"

[1] "t g g" "g t g" "g t t" "t t g" "t t t" "t g a"

In [ ]:
#Splicing state
st_splicing_11 = 'S'
cod_splicing_11 = findCodon(codons, 5090, 30179, 30181)
print(length(st_splicing_11))
print(length(cod_splicing_11))

m29 = rbind(st_splicing_11, cod_splicing_11)
print(m29)

[1] 1
[1] 1
                [,1]   
st_splicing_11  "S"    
cod_splicing_11 "a g g"


In [ ]:
#Exon
st_exon_10 = generateStates(30182, 30310, 'E')
cod_exon_10 = findCodon(codons, 5090, 30182, 30310)
print(length(st_exon_10))
print(length(cod_exon_10))

m30 = rbind(st_exon_10, cod_exon_10)
print(m30)

[1] 43
[1] 43
            [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_10  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_10 "t g g" "t g c" "t c c" "a a a" "t t a" "c t a" "c c c" "c a a"
            [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_10  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_10 "c a g" "c t t" "t g g" "t g c" "t c c" "g g a" "a c a" "a c a"
            [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
st_exon_10  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_10 "g c c" "t t c" "t g c" "c c t" "g g a" "g c a" "c a g" "c a t"
            [,25]   [,26]   [,27]   [,28]   [,29]   [,30]   [,31]   [,32]  
st_exon_10  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_10 "c c a" "a t a" "t t c" "t g g" "a g a" "a g t" "g c g" "g a g"
            [,33]   [,34]   [,35]   [,36]   [,37]   [,38]   [,39]   [,40] 

In [ ]:
#Splicing state
st_splicing_12 = 'S'
cod_splicing_12 = findCodon(codons, 5090, 30311, 30313)
print(length(st_splicing_12))
print(length(cod_splicing_12))

m31 = rbind(st_splicing_12, cod_splicing_12)
print(m31)

[1] 1
[1] 1
                [,1]   
st_splicing_12  "S"    
cod_splicing_12 "g g t"


In [ ]:
#Intron
st_intron_10 = generateStates(30314, 34363, 'I')
cod_intron_10 = findCodon(codons, 5090, 30314, 34363)
print(length(st_intron_10))
print(length(cod_intron_10))
head(cod_intron_10)
tail(cod_intron_10)

#m32 = rbind(states32, aux32)
#print(m32)

[1] 1350
[1] 1350


[1] "a a t" "g a c" "t t c" "t c t" "t t a" "t c t"

[1] "g c t" "c t t" "t t c" "t c t" "g a g" "c a g"

In [ ]:
#Exon
st_exon_11 = generateStates(34364, 34471, 'E')
cod_exon_11 = findCodon(codons, 5090, 34364, 34471)
print(length(st_exon_11))
print(length(cod_exon_11))

m33 = rbind(st_exon_11, cod_exon_11)
print(m33)

[1] 36
[1] 36
            [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_11  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_11 "g t g" "c g g" "g c a" "t t c" "t a t" "g t g" "a a c" "g t g"
            [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_11  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_11 "c t g" "a a t" "g a g" "g a a" "c a g" "a g g" "a a a" "c g t"
            [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
st_exon_11  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_11 "c t g" "t g t" "g a g" "a a c" "a t t" "g c c" "g g c" "c a c"
            [,25]   [,26]   [,27]   [,28]   [,29]   [,30]   [,31]   [,32]  
st_exon_11  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_11 "c t g" "a a g" "g a t" "g c a" "c a a" "a t t" "t t c" "a t c"
            [,33]   [,34]   [,35]   [,36]  
st_exon_11  "E"     "E"     "E

In [ ]:
#Intron
st_intron_11 = generateStates(34472, 37033, 'I')
cod_intron_11 = findCodon(codons, 5090, 34472, 37033)
print(length(st_intron_11))
print(length(cod_intron_11))
head(cod_intron_11)
tail(cod_intron_11)

#m33 = rbind(states33, aux33)
#print(m33)

[1] 854
[1] 854


[1] "g t g" "a g t" "c t t" "t g t" "a a g" "c t g"

[1] "a t t" "t t c" "c t t" "t g g" "c c t" "t a g"

In [ ]:
#Exon
st_exon_12 = generateStates(37034, 37117, 'E')
cod_exon_12 = findCodon(codons, 5090, 37034, 37117)
print(length(st_exon_12))
print(length(cod_exon_12))
#head(aux35)
#tail(aux35)

m35 = rbind(st_exon_12, cod_exon_12)
print(m35)

[1] 28
[1] 28
            [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_12  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_12 "g t c" "a a g" "a a c" "t t c" "a c t" "g a g" "g t c" "c a c"
            [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_12  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_12 "c c t" "g a c" "t a c" "g g g" "a g c" "c a c" "a t c" "c a g"
            [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
st_exon_12  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_12 "g c t" "c t t" "c t g" "g a c" "a a g" "t a c" "a a t" "g c t"
            [,25]   [,26]   [,27]   [,28]  
st_exon_12  "E"     "E"     "E"     "E"    
cod_exon_12 "g a g" "a a g" "c c t" "a a g"


In [ ]:
#Intron
st_intron_12 = generateStates(37118, 37441, 'I')
cod_intron_12 = findCodon(codons, 5090, 37118, 37441)
print(length(st_intron_12))
print(length(cod_intron_12))
head(cod_intron_12)
tail(cod_intron_12)

#m35 = rbind(states35, aux35)
#print(m35)

[1] 108
[1] 108


[1] "g t a" "a g c" "t g g" "g a g" "c a g" "c c t"

[1] "a t c" "t t g" "t t c" "t t t" "t a a" "a a c"

In [ ]:
#Splicing state
st_splicing_13 = 'S'
cod_splicing_13 = findCodon(codons, 5090, 37442, 37444)

m36 = rbind(st_splicing_13, cod_splicing_13)
print(m36)

                [,1]   
st_splicing_13  "S"    
cod_splicing_13 "a g a"


In [ ]:
#Exon
#Finaliza en TGA
st_exon_13 = generateStates(37445, idx_tga, 'E')
cod_exon_13 = findCodon(codons, 5090, 37445, idx_tga)
print(length(st_exon_13))
print(length(cod_exon_13))

m37 = rbind(st_exon_13, cod_exon_13)
print(m37)

[1] 22
[1] 22
            [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
st_exon_13  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_13 "a t g" "c g a" "t t c" "a c a" "c c t" "t t g" "t g c" "a g t"
            [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
st_exon_13  "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_13 "c c g" "g a t" "c t c" "a c t" "t g g" "c g g" "c a a" "g g g"
            [,17]   [,18]   [,19]   [,20]   [,21]   [,22]  
st_exon_13  "E"     "E"     "E"     "E"     "E"     "E"    
cod_exon_13 "a g a" "a g g" "c a a" "a t c" "t g t" "g a g"


In [ ]:
# Entrenamiento Supervisado

total_states = c(st_exon_1, st_intron_1, st_splicing_1, st_exon_2, st_intron_2, st_splicing_2, st_exon_3, st_splicing_3, st_intron_3, st_splicing_4, st_exon_4, st_intron_4,
  st_exon_5, st_intron_5, st_exon_6, st_intron_6, st_splicing_5, st_exon_7, st_splicing_6, st_intron_7, st_splicing_7, st_exon_8, st_splicing_8, st_intron_8, st_splicing_9,
  st_exon_9, st_splicing_10, st_intron_9, st_splicing_11, st_exon_10, st_splicing_12, st_intron_10, st_exon_11, st_intron_11, st_exon_12, st_intron_12, st_splicing_13, st_exon_13)

print(length(codons))
print(length(total_states))

# Matriz de transición
# Estados
# E exon
# I intron
# S el codon no es intron ni exon, ocurre a veces en los lugares donde se da el splicing
# Las transiciones de estado que se pueden dar son:
# 1) exon-exon
# 2) exon-S
# 3) exon-intron
# 4) intron-exon
# 5) intron-S
# 6) intron-intron
# 7) S-exon
# 8) S-S da 0
# 9) S-intron

# Probabilidad de transición exon a ..

total_EE = sum(length(st_exon_1)-1, length(st_exon_2)-1, length(st_exon_3)-1, length(st_exon_4)-1, length(st_exon_5)-1,
  length(st_exon_6)-1, length(st_exon_7)-1, length(st_exon_8)-1, length(st_exon_9)-1, length(st_exon_10)-1, length(st_exon_11)-1,
  length(st_exon_12)-1, length(st_exon_13)-1)

# Exon a S
total_ES = 5

# Exon a Intron
total_EI = 7

total_E = sum(total_EE, total_ES, total_EI)

#print(paste("total_EE = ", total_EE))
#print(paste("total E = ", total_E))

prob_EE = total_EE / total_E
#print(prob_EE)
prob_ES = total_EI / total_E
#print(prob_ES)
prob_EI = total_EI / total_E
#print(prob_EI)


# Probabilidad de transición S a ..
total_SE = 8
total_SS = 0
total_SI = 5
total_S = 13

prob_SE = total_SE / total_S
#print(prob_SE)
prob_SS = total_SS / total_S
#print(prob_SS)
prob_SI = total_SI / total_S
#print(prob_SI)

# Probabilidad de transición intrón a ..
total_IE = 4
total_IS = 8
total_II = sum(length(st_intron_1)-1, length(st_intron_2)-1, length(st_intron_3)-1, length(st_intron_4)-1,
  length(st_intron_5)-1, length(st_intron_6)-1, length(st_intron_7)-1, length(st_intron_8)-1, length(st_intron_9)-1,
  length(st_intron_10)-1, length(st_intron_11)-1, length(st_intron_12)-1)
total_I = sum(total_IE, total_IS, total_II)

print(paste("total_II = ", total_II))
print(paste("total_I = ", total_I))

prob_IE = total_IE / total_I
#print(prob_IE)
prob_IS = total_IS / total_I
#print(prob_IS)
prob_II = total_II / total_I
#print(prob_II)

transition_matrix = matrix(c(prob_EE, prob_SE, prob_IE, prob_ES, prob_SS, prob_IS, prob_EI, prob_SI,
prob_II), 3)
print("Transition matrix")
print(transition_matrix)

[1] 10807
[1] 10807
[1] "total_II =  10260"
[1] "total_I =  10272"
[1] "Transition matrix"
             [,1]         [,2]      [,3]
[1,] 0.9769673704 0.0134357006 0.0134357
[2,] 0.6153846154 0.0000000000 0.3846154
[3,] 0.0003894081 0.0007788162 0.9988318


In [ ]:
# Todos los exones, secuencia de tripletes
cod_exons = c(cod_exon_1, cod_exon_2, cod_exon_3, cod_exon_4, cod_exon_5, cod_exon_6, cod_exon_7, cod_exon_8, cod_exon_9,
  cod_exon_10, cod_exon_11, cod_exon_12, cod_exon_13)


# Probabilidad de emisión en los exones
probs_cods_exons = generate_emission_prob(cod_exons)

            [,1]                 [,2]                 [,3]                
dif_cod     "aaa"                "aac"                "aag"               
vector_prob "0.0191570881226054" "0.0153256704980843" "0.0363984674329502"
            [,4]                 [,5]                 [,6]                
dif_cod     "aat"                "aca"                "acc"               
vector_prob "0.0134099616858238" "0.0229885057471264" "0.0229885057471264"
            [,7]                  [,8]                 [,9]                
dif_cod     "acg"                 "act"                "aga"               
vector_prob "0.00191570881226054" "0.0153256704980843" "0.0114942528735632"
            [,10]                 [,11]                [,12]                
dif_cod     "agc"                 "agg"                "agt"                
vector_prob "0.00957854406130268" "0.0210727969348659" "0.00957854406130268"
            [,13]                 [,14]                [,15]               
dif_cod     "at

In [ ]:
# Probabilidad de emisión en los intrones
cod_introns = c(cod_intron_1, cod_exon_2, cod_intron_3, cod_intron_3, cod_intron_4, cod_intron_5,
  cod_intron_6, cod_intron_7, cod_intron_8, cod_intron_9, cod_intron_10, cod_intron_11, cod_intron_12)

probs_cods_intron = generate_emission_prob(cod_introns)



            [,1]                 [,2]                 [,3]               
dif_cod     "aaa"                "aac"                "aag"              
vector_prob "0.0347078018392169" "0.0113715020270938" "0.017996637990705"
            [,4]                 [,5]                 [,6]                 
dif_cod     "aat"                "aca"                "acc"                
vector_prob "0.0271927222387027" "0.0156234549589637" "0.00919608424799763"
            [,7]                   [,8]                 [,9]                
dif_cod     "acg"                  "act"                "aga"               
vector_prob "0.000988826263225551" "0.0158212202116088" "0.0212597646593494"
            [,10]                [,11]                [,12]               
dif_cod     "agc"                "agg"                "agt"               
vector_prob "0.0132502719272224" "0.0186888163749629" "0.0191832295065757"
            [,13]                [,14]               [,15]               
dif_cod     "ata"   

In [ ]:
cod_splicing_zone = c(cod_splicing_1, cod_splicing_2, cod_splicing_3, cod_splicing_4, cod_splicing_5,
  cod_splicing_6, cod_splicing_7, cod_splicing_8, cod_splicing_9, cod_splicing_10, cod_splicing_11,
  cod_splicing_12, cod_splicing_13)

prob_cods_splicing_zone = generate_emission_prob(cod_splicing_zone)


            [,1]  [,2]  [,3]  [,4]  [,5]  [,6]  [,7]  [,8]  [,9]               
dif_cod     "aaa" "aac" "aag" "aat" "aca" "acc" "acg" "act" "aga"              
vector_prob "0"   "0"   "0"   "0"   "0"   "0"   "0"   "0"   "0.230769230769231"
            [,10] [,11]               [,12] [,13] [,14] [,15] [,16] [,17] [,18]
dif_cod     "agc" "agg"               "agt" "ata" "atc" "atg" "att" "caa" "cac"
vector_prob "0"   "0.307692307692308" "0"   "0"   "0"   "0"   "0"   "0"   "0"  
            [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26] [,27] [,28] [,29]
dif_cod     "cag" "cat" "cca" "ccc" "ccg" "cct" "cga" "cgc" "cgg" "cgt" "cta"
vector_prob "0"   "0"   "0"   "0"   "0"   "0"   "0"   "0"   "0"   "0"   "0"  
            [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38] [,39]
dif_cod     "ctc" "ctg" "ctt" "gaa" "gac" "gag" "gat" "gca" "gcc" "gcg"
vector_prob "0"   "0"   "0"   "0"   "0"   "0"   "0"   "0"   "0"   "0"  
            [,40]                [,41] [,42] [,43]              
dif_c

In [ ]:
emission_matrix = generate_emission_matrix(probs_cods_exons, prob_cods_splicing_zone, probs_cods_intron)

           [,1]       [,2]       [,3]       [,4]       [,5]        [,6]
[1,] 0.01915709 0.01532567 0.03639847 0.01340996 0.02298851 0.022988506
[2,] 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.000000000
[3,] 0.03470780 0.01137150 0.01799664 0.02719272 0.01562345 0.009196084
             [,7]       [,8]       [,9]       [,10]      [,11]       [,12]
[1,] 0.0019157088 0.01532567 0.01149425 0.009578544 0.02107280 0.009578544
[2,] 0.0000000000 0.00000000 0.23076923 0.000000000 0.30769231 0.000000000
[3,] 0.0009888263 0.01582122 0.02125976 0.013250272 0.01868882 0.019183230
           [,13]      [,14]      [,15]      [,16]      [,17]      [,18]
[1,] 0.003831418 0.02298851 0.02298851 0.01724138 0.02107280 0.01724138
[2,] 0.000000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000
[3,] 0.017403342 0.01156927 0.01760111 0.02897261 0.01473351 0.01087709
          [,19]      [,20]      [,21]      [,22]       [,23]      [,24]
[1,] 0.03448276 0.02298851 0.01915709 0.01724138 0.0

In [ ]:
# Verifico que la suma de las probabilidades de cada fila 1
rowSums(emission_matrix)

[1] 1 1 1

In [ ]:
dif_cod_espaciado <- c(
  "a a a", "a a c", "a a g", "a a t",
  "a c a", "a c c", "a c g", "a c t",
  "a g a", "a g c", "a g g", "a g t",
  "a t a", "a t c", "a t g", "a t t",
  "c a a", "c a c", "c a g", "c a t",
  "c c a", "c c c", "c c g", "c c t",
  "c g a", "c g c", "c g g", "c g t",
  "c t a", "c t c", "c t g", "c t t",
  "g a a", "g a c", "g a g", "g a t",
  "g c a", "g c c", "g c g", "g c t",
  "g g a", "g g c", "g g g", "g g t",
  "g t a", "g t c", "g t g", "g t t",
  "t a a", "t a c", "t a g", "t a t",
  "t c a", "t c c", "t c g", "t c t",
  "t g a", "t g c", "t g g", "t g t",
  "t t a", "t t c", "t t g", "t t t"
)


# Modelo oculto de Markov
hmm = initHMM(c("E", "S", "I"), dif_cod_espaciado,
  startProbs = c(1,0,0),
  transProbs = transition_matrix,
  emissionProbs = emission_matrix)

rowSums(hmm$transProbs)
rowSums(hmm$emissionProbs)
print(hmm)

E        S        I 
1.003839 1.000000 1.000000

E S I 
1 1 1

$States
[1] "E" "S" "I"

$Symbols
 [1] "a a a" "a a c" "a a g" "a a t" "a c a" "a c c" "a c g" "a c t" "a g a"
[10] "a g c" "a g g" "a g t" "a t a" "a t c" "a t g" "a t t" "c a a" "c a c"
[19] "c a g" "c a t" "c c a" "c c c" "c c g" "c c t" "c g a" "c g c" "c g g"
[28] "c g t" "c t a" "c t c" "c t g" "c t t" "g a a" "g a c" "g a g" "g a t"
[37] "g c a" "g c c" "g c g" "g c t" "g g a" "g g c" "g g g" "g g t" "g t a"
[46] "g t c" "g t g" "g t t" "t a a" "t a c" "t a g" "t a t" "t c a" "t c c"
[55] "t c g" "t c t" "t g a" "t g c" "t g g" "t g t" "t t a" "t t c" "t t g"
[64] "t t t"

$startProbs
E S I 
1 0 0 

$transProbs
    to
from            E            S         I
   E 0.9769673704 0.0134357006 0.0134357
   S 0.6153846154 0.0000000000 0.3846154
   I 0.0003894081 0.0007788162 0.9988318

$emissionProbs
      symbols
states      a a a      a a c      a a g      a a t      a c a       a c c
     E 0.01915709 0.01532567 0.03639847 0.01340996 0.02298851 0.022988506
     S 0.00000000 0.00000

In [ ]:
#Viterbi
states = viterbi(hmm, c(codons))
v = rbind(states, c(codons))
print(v)

       [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]    [,9]   
states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
       "a t g" "g c t" "g a c" "a g c" "c g g" "g a t" "c c c" "g c c" "a g c"
       [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]   [,17]   [,18]  
states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
       "g a c" "c a g" "a t g" "c a g" "c a c" "t g g" "a a g" "g a g" "c a g"
       [,19]   [,20]   [,21]   [,22]   [,23]   [,24]   [,25]   [,26]   [,27]  
states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
       "c g g" "g c c" "g c g" "c a g" "g t a" "c a c" "t c t" "g t g" "c t c"
       [,28]   [,29]   [,30]   [,31]   [,32]   [,33]   [,34]   [,35]   [,36]  
states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
       "c c c" "g a g" "c g g" "g c c" "c g a" "a g g" "t c c" "g t t" "t a g"
       [,37]   [,38]   [,39]   [,40]   [,41]   [,42]

** Problema al entrar en intron nunca vuelve a cambiar de estado

In [ ]:
# Baum-Welch
bw = baumWelch(hmm, codons, maxIteration=10)
print(bw$hmm)

$States
[1] "E" "S" "I"

$Symbols
 [1] "a a a" "a a c" "a a g" "a a t" "a c a" "a c c" "a c g" "a c t" "a g a"
[10] "a g c" "a g g" "a g t" "a t a" "a t c" "a t g" "a t t" "c a a" "c a c"
[19] "c a g" "c a t" "c c a" "c c c" "c c g" "c c t" "c g a" "c g c" "c g g"
[28] "c g t" "c t a" "c t c" "c t g" "c t t" "g a a" "g a c" "g a g" "g a t"
[37] "g c a" "g c c" "g c g" "g c t" "g g a" "g g c" "g g g" "g g t" "g t a"
[46] "g t c" "g t g" "g t t" "t a a" "t a c" "t a g" "t a t" "t c a" "t c c"
[55] "t c g" "t c t" "t g a" "t g c" "t g g" "t g t" "t t a" "t t c" "t t g"
[64] "t t t"

$startProbs
E S I 
1 0 0 

$transProbs
    to
from          E           S          I
   E 0.94427573 0.017773118 0.03795115
   S 0.91523536 0.000000000 0.08476464
   I 0.00746355 0.008926806 0.98360964

$emissionProbs
      symbols
states       a a a      a a c      a a g       a a t      a c a       a c c
     E 0.009921045 0.01174011 0.01325623 0.008873473 0.01470072 0.013863389
     S 0.000000000 0.00000000

In [ ]:
states2 = viterbi(bw$hmm, codons)
v2 = rbind(states2, codons)
print(v2)

        [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]    [,9]   
states2 "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
codons  "a t g" "g c t" "g a c" "a g c" "c g g" "g a t" "c c c" "g c c" "a g c"
        [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]   [,17]   [,18]  
states2 "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
codons  "g a c" "c a g" "a t g" "c a g" "c a c" "t g g" "a a g" "g a g" "c a g"
        [,19]   [,20]   [,21]   [,22]   [,23]   [,24]   [,25]   [,26]   [,27]  
states2 "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
codons  "c g g" "g c c" "g c g" "c a g" "g t a" "c a c" "t c t" "g t g" "c t c"
        [,28]   [,29]   [,30]   [,31]   [,32]   [,33]   [,34]   [,35]   [,36]  
states2 "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
codons  "c c c" "g a g" "c g g" "g c c" "c g a" "a g g" "t c c" "g t t" "t a g"
        [,37]   [,38]   [,39]   [,40]   

** Por lo menos no se queda atorado en el estado I, pero cambia mucho la matriz de transición, fila S

In [ ]:
comparativa = rbind(total_states, states2, codons)
print(comparativa)

             [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
total_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
states2      "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
codons       "a t g" "g c t" "g a c" "a g c" "c g g" "g a t" "c c c" "g c c"
             [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
total_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
states2      "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
codons       "a g c" "g a c" "c a g" "a t g" "c a g" "c a c" "t g g" "a a g"
             [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
total_states "E"     "E"     "E"     "E"     "E"     "E"     "I"     "I"    
states2      "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
codons       "g a g" "c a g" "c g g" "g c c" "g c g" "c a g" "g t a" "c a c"
             [,25]   [,26]   [,27]   [,28]   [,29]   [,30]   [,31]   [,32]  

**************************************************
**************************************************
Como los resultados que me devuelve el Viterbi no es el esperado, decidí ampliar los tripletes que le asigno a la zona de Splicing.

In [ ]:
##########
# Exones #
##########

exons_start = c(5090, 15268, 17064, 18153, 19166, 19877, 22087, 22741, 27327, 30181, 34364, 37034, 37444)
exons_end = c(5155, 15439, 17174, 18283, 19270, 20002, 22278, 22893, 27465, 30311, 34471, 37117, 37510)

# Modifico los indices de comienzo y fin de los exones, para aumentar los tripletes de la zona de splicing

new_exons_start = sum_index(exons_start, 1, 5090)
new_exons_end = rest_index(exons_end, 1, 5090)

result = extract_sec(new_exons_start, new_exons_end, 'E')
all_exons_states = result$states
all_exons_codons = result$triplets

# Probabilidades de transición de exon a ..
total_EE = length(all_exons_states) - 13       # las transiciones son uno menos que la cantidad de elementos y como son 13 exones le resto 13

# Exon a S
total_ES = 12

# Exon a Intron
total_EI = 0    # Ahora si o si pasa por la zona de Splicing por lo que yya no habrá transiciones E a I

total_E = sum(total_EE, total_ES, total_EI)

print("Nueva secuencias de comienzo de exones: ")
print(new_exons_start)
print("Nueva secuencias de fin de exones: ")
print(new_exons_end)

prob_EE = total_EE / total_E
prob_ES = total_ES / total_E
prob_EI = total_EI / total_E
cat(paste("\nProbabilidad de transicionar de un exón a otro exón", prob_EE))
cat(paste("\nProbabilidad de transicionar de un exón a splicing zone", prob_ES))




                      [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]   
states_all_secuencies "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_all_secuencies    "g c t" "g a c" "a g c" "c g g" "g a t" "c c c" "g c c"
                      [,8]    [,9]    [,10]   [,11]   [,12]   [,13]   [,14]  
states_all_secuencies "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_all_secuencies    "a g c" "g a c" "c a g" "a t g" "c a g" "c a c" "t g g"
                      [,15]   [,16]   [,17]   [,18]   [,19]   [,20]   [,21]  
states_all_secuencies "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_all_secuencies    "a a g" "g a g" "c a g" "c g g" "g c c" "g c g" "a a g"
                      [,22]   [,23]   [,24]   [,25]   [,26]   [,27]   [,28]  
states_all_secuencies "E"     "E"     "E"     "E"     "E"     "E"     "E"    
cod_all_secuencies    "c t g" "a t g" "t c c" "t g a" "c c a" "c t g" "g a g"
                      [,29]   [,30]   [,31]   [,32]   [,33]   [,

In [ ]:
####################
# Zona de Splicing #
####################

splicing_zone_len = 6

# Unión E - I
splicing_start_EI = c()
splicing_end_EI = c()

for(i in new_exons_end) {
  splicing_start_EI = c(splicing_start_EI, sum(i,1))
}

for(i in splicing_start_EI) {
  splicing_end_EI = c(splicing_end_EI, sum(i, splicing_zone_len))
}

# Unión I - E
splicing_start_IE = c()
splicing_end_IE = c()

new_exons_start_2 = new_exons_start[2:length(new_exons_start)]
for(i in new_exons_start_2) {
  splicing_end_IE = c(splicing_end_IE, (i - 1))
}

for(i in splicing_end_IE) {
  splicing_start_IE = c(splicing_start_IE, (i - splicing_zone_len))
}

splicing_start = c()
splicing_end = c()

for(i in 1:length(splicing_start_IE)) {
  splicing_start = c(splicing_start, splicing_start_EI[i], splicing_start_IE[i])
}
print("Splicing start conjunto")
print(splicing_start)

for(i in 1:length(splicing_end_IE)) {
  splicing_end = c(splicing_end, splicing_end_EI[i], splicing_end_IE[i])
}
print("Splicing end conjunto")
print(splicing_end)

result_spl = extract_sec(splicing_start, splicing_end, 'S')
all_splicing_states = result_spl$states
all_splicing_triplets = result_spl$triplets

# Probabilidades de transición de Splicing Zone a ...
total_SE = 12
total_SI = 12
total_SS = length(all_splicing_states) - 24       # 24 es la acantidad total de splicing zones
total_S =  sum(total_SE, total_SI, total_SS)

prob_SE = total_SE / total_S
prob_SI = total_SI / total_S
prob_SS = total_SS / total_S

cat(paste("\nProbabilidad de transicionar de S a exón", prob_SE))
cat(paste("\nProbabilidad de transicionar de S a S", prob_SS))
cat(paste("\nProbabilidad de transicionar de S a intrón", prob_SI))

[1] "Splicing start conjunto"
 [1]  5153 15264 15437 17061 17171 18150 18281 19164 19268 19875 20000 22083
[13] 22274 22737 22889 27324 27461 30177 30308 34362 34469 37032 37115 37440
[1] "Splicing end conjunto"
 [1]  5159 15270 15443 17067 17177 18156 18287 19170 19274 19881 20006 22089
[13] 22280 22743 22895 27330 27467 30183 30314 34368 34475 37038 37121 37446
                      [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]   
states_all_secuencies "S"     "S"     "S"     "S"     "S"     "S"     "S"    
cod_all_secuencies    "c a g" "g t a" "c a c" "t g t" "a g a" "a a g" "c a g"
                      [,8]    [,9]    [,10]   [,11]   [,12]   [,13]   [,14]  
states_all_secuencies "S"     "S"     "S"     "S"     "S"     "S"     "S"    
cod_all_secuencies    "g t a" "a g t" "t t a" "g g g" "g c c" "g t t" "g g t"
                      [,15]   [,16]   [,17]   [,18]   [,19]   [,20]   [,21]  
states_all_secuencies "S"     "S"     "S"     "S"     "S"     "S"     "S"    
cod_all_se

In [ ]:
############
# Intrones #
############
intron_start = c()
intron_end = c()

for(i in splicing_end_EI) {
  intron_start = c(intron_start, sum(i,1))
}

### Corrección
intron_start = intron_start[1:(length(intron_start) - 1)]   #La secuencia termina en exón no va haber un nuevo intrón
###

print("Vector de comienzos de intrones: ")
print(intron_start)

for(i in splicing_start_IE) {
  intron_end = c(intron_end, (i - 1))
}
print("Vector de finalización de intrones: ")
print(intron_end)

result_intro = extract_sec(intron_start, intron_end, 'I')
all_introns_states = result_intro$states
all_introns_triplets = result_intro$triplets


# Probabilidades de transición de intron a ...
# Intrón a intrón
total_II = length(all_introns_states) - 12

# Intrón a splicing zone
total_IS = 12

# Intrón a exón
total_IE = 0

total_I = sum(total_II, total_IS, total_IE)

prob_II = total_II / total_I
prob_IS = total_IS / total_I
prob_IE = total_IE / total_I

cat(paste("\nProbabilidad de transicionar de intrón a intrón", prob_II))
cat(paste("\nProbabilidad de transicionar de intrón a S", prob_IS))
cat(paste("\nProbabilidad de transicionar de intrón a exón", prob_IE))


[1] "Vector de comienzos de intrones: "
 [1]  5160 15444 17178 18288 19275 20007 22281 22896 27468 30315 34476 37122
[1] "Vector de finalización de intrones: "
 [1] 15263 17060 18149 19163 19874 22082 22736 27323 30176 34361 37031 37439
                      [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]   
states_all_secuencies "I"     "I"     "I"     "I"     "I"     "I"     "I"    
cod_all_secuencies    "c a c" "t c t" "g t g" "c t c" "c c c" "g a g" "c g g"
                      [,8]    [,9]    [,10]   [,11]   [,12]   [,13]   [,14]  
states_all_secuencies "I"     "I"     "I"     "I"     "I"     "I"     "I"    
cod_all_secuencies    "g c c" "c g a" "a g g" "t c c" "g t t" "t a g" "a a a"
                      [,15]   [,16]   [,17]   [,18]   [,19]   [,20]   [,21]  
states_all_secuencies "I"     "I"     "I"     "I"     "I"     "I"     "I"    
cod_all_secuencies    "g c g" "g g g" "g c g" "t c g" "g c a" "a g t" "a a a"
                      [,22]   [,23]   [,24]   [,25]   [,26]  

In [ ]:
# Matriz de transicion
transition_matrix_2 = matrix(c(prob_EE, prob_SE, prob_IE, prob_ES, prob_SS, prob_IS, prob_EI, prob_SI,
prob_II), 3)
print(transition_matrix_2)

          [,1]        [,2]      [,3]
[1,] 0.9761431 0.023856859 0.0000000
[2,] 0.1666667 0.666666667 0.1666667
[3,] 0.0000000 0.001170389 0.9988296


In [ ]:
# Matriz de emisión
probs_cods_exons = generate_emission_prob(all_exons_codons)
probs_cods_splicing_zone = generate_emission_prob(all_splicing_triplets)
probs_cods_intron = generate_emission_prob(all_introns_triplets)

emission_matrix_2 = generate_emission_matrix(probs_cods_exons, prob_cods_splicing_zone, probs_cods_intron)

            [,1]                 [,2]                 [,3]                
dif_cod     "aaa"                "aac"                "aag"               
vector_prob "0.0198412698412698" "0.0158730158730159" "0.0337301587301587"
            [,4]                 [,5]                 [,6]                
dif_cod     "aat"                "aca"                "acc"               
vector_prob "0.0138888888888889" "0.0238095238095238" "0.0238095238095238"
            [,7]                  [,8]                 [,9]                
dif_cod     "acg"                 "act"                "aga"               
vector_prob "0.00198412698412698" "0.0158730158730159" "0.0119047619047619"
            [,10]                 [,11]                [,12]                
dif_cod     "agc"                 "agg"                "agt"                
vector_prob "0.00992063492063492" "0.0218253968253968" "0.00992063492063492"
            [,13]                 [,14]                [,15]               
dif_cod     "at

In [ ]:
rowSums(emission_matrix_2)

[1] 1 1 1

In [ ]:
# Modelo oculto de Markov
hmm2 = initHMM(c("E", "S", "I"), dif_cod_espaciado,
  startProbs = c(1,0,0),
  transProbs = transition_matrix_2,
  emissionProbs = emission_matrix_2)

rowSums(hmm2$transProbs)
rowSums(hmm2$emissionProbs)
print(hmm2)

E S I 
1 1 1

E S I 
1 1 1

$States
[1] "E" "S" "I"

$Symbols
 [1] "a a a" "a a c" "a a g" "a a t" "a c a" "a c c" "a c g" "a c t" "a g a"
[10] "a g c" "a g g" "a g t" "a t a" "a t c" "a t g" "a t t" "c a a" "c a c"
[19] "c a g" "c a t" "c c a" "c c c" "c c g" "c c t" "c g a" "c g c" "c g g"
[28] "c g t" "c t a" "c t c" "c t g" "c t t" "g a a" "g a c" "g a g" "g a t"
[37] "g c a" "g c c" "g c g" "g c t" "g g a" "g g c" "g g g" "g g t" "g t a"
[46] "g t c" "g t g" "g t t" "t a a" "t a c" "t a g" "t a t" "t c a" "t c c"
[55] "t c g" "t c t" "t g a" "t g c" "t g g" "t g t" "t t a" "t t c" "t t g"
[64] "t t t"

$startProbs
E S I 
1 0 0 

$transProbs
    to
from         E           S         I
   E 0.9761431 0.023856859 0.0000000
   S 0.1666667 0.666666667 0.1666667
   I 0.0000000 0.001170389 0.9988296

$emissionProbs
      symbols
states      a a a      a a c      a a g      a a t      a c a       a c c
     E 0.01984127 0.01587302 0.03373016 0.01388889 0.02380952 0.023809524
     S 0.00000000 0.00000000 0.00000000 0

In [ ]:
#Viterbi
states3 = viterbi(hmm2, c(codons))
v3 = rbind(states3, c(codons))
print(v3)

        [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]    [,9]   
states3 "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
        "a t g" "g c t" "g a c" "a g c" "c g g" "g a t" "c c c" "g c c" "a g c"
        [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]   [,17]   [,18]  
states3 "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
        "g a c" "c a g" "a t g" "c a g" "c a c" "t g g" "a a g" "g a g" "c a g"
        [,19]   [,20]   [,21]   [,22]   [,23]   [,24]   [,25]   [,26]   [,27]  
states3 "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
        "c g g" "g c c" "g c g" "c a g" "g t a" "c a c" "t c t" "g t g" "c t c"
        [,28]   [,29]   [,30]   [,31]   [,32]   [,33]   [,34]   [,35]   [,36]  
states3 "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
        "c c c" "g a g" "c g g" "g c c" "c g a" "a g g" "t c c" "g t t" "t a g"
        [,37]   [,38]   [,39]   [,40]   

In [ ]:
# Baum-Welch
bw2 = baumWelch(hmm2, codons, maxIteration=100)
print(bw2$hmm)

$States
[1] "E" "S" "I"

$Symbols
 [1] "a a a" "a a c" "a a g" "a a t" "a c a" "a c c" "a c g" "a c t" "a g a"
[10] "a g c" "a g g" "a g t" "a t a" "a t c" "a t g" "a t t" "c a a" "c a c"
[19] "c a g" "c a t" "c c a" "c c c" "c c g" "c c t" "c g a" "c g c" "c g g"
[28] "c g t" "c t a" "c t c" "c t g" "c t t" "g a a" "g a c" "g a g" "g a t"
[37] "g c a" "g c c" "g c g" "g c t" "g g a" "g g c" "g g g" "g g t" "g t a"
[46] "g t c" "g t g" "g t t" "t a a" "t a c" "t a g" "t a t" "t c a" "t c c"
[55] "t c g" "t c t" "t g a" "t g c" "t g g" "t g t" "t t a" "t t c" "t t g"
[64] "t t t"

$startProbs
E S I 
1 0 0 

$transProbs
    to
from         E          S         I
   E 0.9174926 0.08250745 0.0000000
   S 0.5823693 0.09310817 0.3245225
   I 0.0000000 0.01829279 0.9817072

$emissionProbs
      symbols
states      a a a      a a c      a a g      a a t      a c a       a c c
     E 0.01454826 0.01285658 0.01509564 0.01305371 0.01266252 0.013230843
     S 0.00000000 0.00000000 0.00000000 0.000

In [ ]:
#Viterbi
states4 = viterbi(bw2$hmm, c(codons))
v4 = rbind(states4, c(codons))
print(v4)

        [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]    [,9]   
states4 "E"     "S"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
        "a t g" "g c t" "g a c" "a g c" "c g g" "g a t" "c c c" "g c c" "a g c"
        [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]   [,17]   [,18]  
states4 "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
        "g a c" "c a g" "a t g" "c a g" "c a c" "t g g" "a a g" "g a g" "c a g"
        [,19]   [,20]   [,21]   [,22]   [,23]   [,24]   [,25]   [,26]   [,27]  
states4 "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
        "c g g" "g c c" "g c g" "c a g" "g t a" "c a c" "t c t" "g t g" "c t c"
        [,28]   [,29]   [,30]   [,31]   [,32]   [,33]   [,34]   [,35]   [,36]  
states4 "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
        "c c c" "g a g" "c g g" "g c c" "c g a" "a g g" "t c c" "g t t" "t a g"
        [,37]   [,38]   [,39]   [,40]   

In [ ]:
comparativa2 = rbind(total_states, states4, codons)
print(comparativa2)

             [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
total_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
states4      "E"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
codons       "a t g" "g c t" "g a c" "a g c" "c g g" "g a t" "c c c" "g c c"
             [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
total_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
states4      "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
codons       "a g c" "g a c" "c a g" "a t g" "c a g" "c a c" "t g g" "a a g"
             [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
total_states "E"     "E"     "E"     "E"     "E"     "E"     "I"     "I"    
states4      "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
codons       "g a g" "c a g" "c g g" "g c c" "g c g" "c a g" "g t a" "c a c"
             [,25]   [,26]   [,27]   [,28]   [,29]   [,30]   [,31]   [,32]  

In [ ]:
# Creo que funciona mejor que antes, pero le falta precisión, puede estar adelantado o atrasado unos codones
# le pega a la mayoría de los exones, no a todos
# Detecta exones donde no debería haber

# Buscar zonas de exon en la secuencia de tripletes
idx_to_codon_idx <- function(idx) {
  new_idx = (idx - 5090) / 3
  return (floor(new_idx))
}

compare <- function(gap, start, qtty, states, viterbi_states, triplets) {

  start_aux = floor((start - gap) / 3)
  end_aux = start_aux + qtty

  states = states[start_aux:end_aux]
  viterbi_states = viterbi_states[start_aux:end_aux]
  triplets = triplets[start_aux:end_aux]

  comparative = rbind(states, viterbi_states, triplets)

  return (comparative)
}

exon2 = compare(5090, 15268, 50, total_states, states4, codons)
print(exon2)


               [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
states         "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "t g t" "a g a" "a a g" "c t g" "a t g" "t c c" "t g a" "c c a"
               [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "c t g" "g a g" "c t g" "g t a" "a c c" "c a g" "t a g" "g a g"
               [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "a c a" "a a c" "t t a" "a t g" "t t a" "t t a" "c a g" "t a g"
               [,25]   [,26]   [,27]   [,28]   [,29]

In [ ]:
exon3 = compare(5090, 17064, 50, total_states, states4, codons)
print(exon3)

               [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
states         "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "I"     "I"     "I"     "I"     "I"     "I"     "I"    
triplets       "t t a" "g g g" "g c c" "t t t" "g g c" "t a c" "t t t" "g a g"
               [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "I"     "I"     "I"     "I"     "I"     "I"     "I"    
triplets       "g t c" "a c a" "c a t" "g a c" "a t t" "a c c" "a a a" "t a c"
               [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "I"     "I"     "I"     "I"     "I"     "I"     "I"    
triplets       "t c c" "a a g" "g c a" "a a g" "g t a" "t t t" "g a g" "c a t"
               [,25]   [,26]   [,27]   [,28]   [,29]

In [ ]:
exon4 = compare(5090, 18153, 50, total_states, states4, codons)
print(exon4)

               [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
states         "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "S"     "E"     "E"     "E"     "S"     "E"     "S"    
triplets       "g t a" "g c t" "g g a" "g a a" "t c g" "g g t" "t c a" "g c t"
               [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "S"    
triplets       "g a c" "a c a" "g t t" "c g g" "g a c" "c c t" "c g t" "g g g"
               [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "I"     "I"     "I"     "I"     "I"     "I"     "I"    
triplets       "t t t" "g c a" "g t g" "a a a" "t t t" "t a c" "a c a" "g a a"
               [,25]   [,26]   [,27]   [,28]   [,29]

In [ ]:
exon5 = compare(5090, 19166, 50, total_states, states4, codons)
print(exon5)

               [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
states         "I"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "I"     "I"     "I"     "I"     "I"     "I"     "I"    
triplets       "t a g" "t t t" "c c a" "t c t" "t t t" "a t c" "c a c" "a g c"
               [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "I"     "S"     "E"     "E"     "E"     "E"     "E"    
triplets       "c a a" "a a g" "a g a" "a a t" "c c t" "c a g" "a c a" "c a t"
               [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "c t g" "a a g" "g a t" "c c g" "g a c" "a t g" "g t c" "t g g"
               [,25]   [,26]   [,27]   [,28]   [,29]

In [ ]:
exon6 = compare(5090, 19877, 50, total_states, states4, codons)
print(exon6)

               [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
states         "I"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "I"     "I"     "I"     "I"     "I"     "I"     "I"    
triplets       "t a g" "g t t" "t c t" "t t c" "t t g" "t t c" "a g t" "g a t"
               [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "I"     "I"     "I"     "I"     "I"     "I"     "I"    
triplets       "c g g" "g g g" "a t t" "c c a" "g a t" "g g a" "c a t" "c g c"
               [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "I"     "I"     "I"     "I"     "I"     "I"     "I"    
triplets       "c a c" "a t g" "a a t" "g g a" "t a t" "g g a" "t c a" "c a t"
               [,25]   [,26]   [,27]   [,28]   [,29]

In [ ]:
exon7 = compare(5090, 22087, 50, total_states, states4, codons)
print(exon7)

               [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
states         "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "t g t" "a g a" "c t g" "a c c" "a g g" "g c a" "t c a" "a a a"
               [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "a c c" "t t t" "c t g" "t t g" "a a g" "a t g" "c g g" "c g a"
               [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "g a c" "t t t" "c c c" "a g g" "a a g" "a t c" "c t g" "a c t"
               [,25]   [,26]   [,27]   [,28]   [,29]

In [ ]:
exon8 = compare(5090, 22741, 50, total_states, states4, codons)
print(exon8)

               [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
states         "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "t t c" "a g g" "t t t" "g g c" "c t c" "a c a" "a g g" "a c t"
               [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "a c c" "c t c" "t c a" "t c c" "c a g" "t t g" "g t a" "a a c"
               [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "t g g" "t c t" "t a a" "a c c" "g g a" "a t c" "c a g" "t t a"
               [,25]   [,26]   [,27]   [,28]   [,29]

In [ ]:
exon9 = compare(5090, 27327, 50, total_states, states4, codons)
print(exon9)

               [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
states         "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "g c a" "g g g" "c c g" "c c t" "t t t" "t g c" "c t a" "t c c"
               [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "t g a" "c a c" "t c a" "c c g" "c c a" "t c g" "c c t" "g g g"
               [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "a c c" "c a a" "t t a" "t c t" "t c a" "t a t" "a c c" "t g t"
               [,25]   [,26]   [,27]   [,28]   [,29]

In [ ]:
exon10 = compare(5090, 30181, 50, total_states, states4, codons)
print(exon10)

               [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
states         "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "t g a" "a g g" "t g g" "t g c" "t c c" "a a a" "t t a" "c t a"
               [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "c c c" "c a a" "c a g" "c t t" "t g g" "t g c" "t c c" "g g a"
               [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "a c a" "a c a" "g c c" "t t c" "t g c" "c c t" "g g a" "g c a"
               [,25]   [,26]   [,27]   [,28]   [,29]

In [ ]:
exon11 = compare(5090, 34364, 50, total_states, states4, codons)
print(exon11)

               [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
states         "I"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "c a g" "g t g" "c g g" "g c a" "t t c" "t a t" "g t g" "a a c"
               [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "g t g" "c t g" "a a t" "g a g" "g a a" "c a g" "a g g" "a a a"
               [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "c g t" "c t g" "t g t" "g a g" "a a c" "a t t" "g c c" "g g c"
               [,25]   [,26]   [,27]   [,28]   [,29]

In [ ]:
exon12 = compare(5090, 37034, 50, total_states, states4, codons)
print(exon12)

               [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
states         "I"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "t a g" "g t c" "a a g" "a a c" "t t c" "a c t" "g a g" "g t c"
               [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "c a c" "c c t" "g a c" "t a c" "g g g" "a g c" "c a c" "a t c"
               [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "c a g" "g c t" "c t t" "c t g" "g a c" "a a g" "t a c" "a a t"
               [,25]   [,26]   [,27]   [,28]   [,29]

In [ ]:
exon13 = compare(5090, 37444, 50, total_states, states4, codons)
print(exon13)

               [,1]    [,2]    [,3]    [,4]    [,5]    [,6]    [,7]    [,8]   
states         "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "I"     "S"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "a a c" "a g a" "a t g" "c g a" "t t c" "a c a" "c c t" "t t g"
               [,9]    [,10]   [,11]   [,12]   [,13]   [,14]   [,15]   [,16]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
triplets       "t g c" "a g t" "c c g" "g a t" "c t c" "a c t" "t g g" "c g g"
               [,17]   [,18]   [,19]   [,20]   [,21]   [,22]   [,23]   [,24]  
states         "E"     "E"     "E"     "E"     "E"     "E"     "E"     "E"    
viterbi_states "E"     "S"     "S"     "S"     "E"     "E"     "E"     "E"    
triplets       "c a a" "g g g" "a g a" "a g g" "c a a" "a t c" "t g t" "g a g"
               [,25] [,26] [,27] [,28] [,29] [,30] [

Para mejorar los resultados quzás se podría agregar otro estado más:

*   S1: para la zona de splicing E-I
*   S2: para la zona de splicing I-E

Así quedarían 4 estados E, I, S1 y S2, y cómo señale anteriormente podríamos jugar con el tamaño de la zona que abarcarían S1 y S2.

